In [6]:
# from google.colab import drive
# import os
# import sys
# drive.mount('/content/drive')
# loc_prefix = '/content/drive/My Drive/dl/comp1'

# # Import Custom Library Path
# sys.path.append(loc_prefix)

In [7]:
# !pip install Pattern
# Or
# !git clone https://github.com/clips/pattern.git
# !cd pattern
# !python ./pattern/setup.py install

In [5]:
import pandas as pd
import numpy as np
from IPython.display import display

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [6]:
from lib.tune import Tune

paral_procs = -1
tune = Tune()
loc_prefix = '.'

def save_to(df, file_name):
    df.to_csv(loc_prefix + '/data/' + file_name, index=False)
def read_csv(file_name):
    return pd.read_csv(loc_prefix + '/data/' + file_name)

In [7]:
def read_train_test(rel_path_train, rel_path_test):
    dfs = []
    for path in [rel_path_train, rel_path_test]:
        df = read_csv(path)
        print(f'{path} {df.shape}')
        display(df.head(5))
        
        dfs.append(df)
    
    return dfs[0], dfs[1]

In [11]:
# df = pd.read_csv(loc_prefix + '/data/feats_merge_train.csv')
df_train, df_test = read_train_test('train/feats_merge_train.csv', 'test/feats_merge_test.csv')

train/feats_merge_train.csv (27643, 20)


,Id,Popularity,Page content,data-channel,title,raw_content,content_length,avg_word_length,topics,author,year,month,day,hour,min,sec,weekends,img_number,link_number,title_length
0,0,-1,"<html><head><div class=""article-info""> <span c...",world,NASA's Grand Challenge: Stop Asteroids From De...,There may be killer asteroids headed for Eart...,583,5.214905,"asteroid,asteroids,challenge,earth,space,u.s.,...",NaN,2013,6,19,15,4,30,0,1,22,60
1,1,1,"<html><head><div class=""article-info""><span cl...",tech,Google's New Open Source Patent Pledge: We Won...,Google took a stand of sorts against patent-l...,309,5.032787,"apps and software,google,open source,opn pledg...",christina,2013,3,28,17,40,55,0,2,18,74
2,2,1,"<html><head><div class=""article-info""><span cl...",entertainment,Ballin': 2014 NFL Draft Picks Get to Choose Th...,You've spend countless hours training to be a...,1360,4.750225,"entertainment,nfl,nfl draft,sports,television",sam-laird,2014,5,7,19,15,20,0,2,11,68
3,3,-1,"<html><head><div class=""article-info""><span cl...",watercooler,Cameraperson Fails Deliver Slapstick Laughs,Tired of the same old sports fails and ne...,476,4.841727,"sports,video,videos,watercooler",sam-laird,2013,10,11,2,26,50,0,1,13,43
4,4,-1,"<html><head><div class=""article-info""><span cl...",entertainment,NFL Star Helps Young Fan Prove Friendship With...,"At 6-foot-5 and 298 pounds, All-Pro NFL star ...",1937,5.089650,"entertainment,instagram,instagram video,nfl,sp...",connor-finnegan,2014,4,17,3,31,43,0,52,16,61


test/feats_merge_test.csv (11847, 19)


,Id,Page content,data-channel,title,raw_content,content_length,avg_word_length,topics,author,year,month,day,hour,min,sec,weekends,img_number,link_number,title_length
0,27643,"<html><head><div class=""article-info""><span cl...",entertainment,Soccer Star Gets Twitter Death Threats After T...,Note to humanity: One Direction fandom ai...,622,5.220114,"entertainment,music,one direction,soccer,sports",sam-laird,2013,9,9,19,47,2,0,1,30,74
1,27644,"<html><head><div class=""article-info""><span cl...",tech,Google Glass Gets an Accessory Store,Shortly after announcing a hardware upgrade f...,149,4.739437,"gadgets,glass,google,google glass,google glass...",stan-schroeder,2013,10,31,9,25,2,0,3,13,36
2,27645,"<html><head><div class=""article-info""><span cl...",business,OUYA Gaming Console Already Sold Out on Amazon,"Well, that was quick. Just hours after going ...",168,5.018293,"amazon,amazon kindle,business,gaming",todd-wasserman,2013,6,25,12,54,54,0,2,13,46
3,27646,"<html><head><div class=""article-info""><span cl...",film,'Between Two Ferns' Mocks Oscar Nominees,Between Two Ferns: Oscar Buzz Edition Part 1...,162,5.581699,"between two ferns,movies,the oscars,oscars 201...",neha-prakash,2013,2,13,3,30,21,0,1,15,40
4,27647,"<html><head><div class=""article-info""><span cl...",entertainment,'American Sniper' Trailer: Looks Like Eastwood...,Ever since The Hurt Locker it seems like ...,225,5.041096,"american sniper,awards,bradley cooper,clint ea...",josh-dickey,2014,10,3,1,34,54,0,1,10,70


# Select Number of Samples

In [12]:
# sel_num = 100
# X_train = X_train[:sel_num]
# y_train = y_train[:sel_num]

# Stemming & Remove Stop Word

In [13]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('wordnet')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    join_str = ' '.join([porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)])
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(join_str)

def tokenizer_lemm_nostop(text):
    lemm = WordNetLemmatizer()
    join_str = ' '.join([lemm.lemmatize(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)])
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(join_str)

def stem_nostop_str(str_array):
    return [' '.join(tokenizer_stem_nostop(row)) for row in str_array]

def lemma_nostop_str(str_array):
    return [' '.join(tokenizer_lemm_nostop(row)) for row in str_array]

print('Porter Stemmed')
print(tokenizer_stem_nostop('runners like running and thus they run.'))
print('Lemmatized')
print(tokenizer_lemm_nostop('runners like running and thus they run.'))

# X_train['title'] = stem_nostop_str(X_train['title'].values)
# X_train['raw_content'] = stem_nostop_str(X_train['raw_content'].values)
# X_train.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pp20/pp20s02/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/pp20/pp20s02/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Porter Stemmed
['runner', 'like', 'run', 'thu', 'run']
Lemmatized
['runner', 'like', 'running', 'thus', 'run']


# Data-Channel & Author LabelEncode & Stemming

In [14]:
from sklearn.preprocessing import LabelEncoder

def preproc(df):
    le = LabelEncoder()
    df['data-channel'] = le.fit_transform(df['data-channel'].astype(str))
    df['author'] = le.fit_transform(df['author'].astype(str))
    df['title'] = stem_nostop_str(df['title'].astype(str))
    df['raw_content'] = stem_nostop_str(df['raw_content'].astype(str))
    df['topics'] = stem_nostop_str(df['topics'].astype(str))
    df = df.reset_index()

    return df

def preporoc_train_test(df_train, df_test):
    df_arr = [df_train, df_test]
    df_arr_preproc = []
    for df in df_arr:
        df_arr_preproc.append(preproc(df))
    return df_arr_preproc[0], df_arr_preproc[1]

In [80]:
df_train, df_test = preporoc_train_test(df_train, df_test)

In [81]:
display(f'df_train {df_train.shape}')
display(df_train.head(5))
display(f'df_test {df_test.shape}')
display(df_test.head(5))

save_to(df_train, f'train/preprocessed_train.csv')
save_to(df_test, f'test/preprocessed_test.csv')

'df_train (27643, 21)'

,index,Id,Popularity,Page content,data-channel,title,raw_content,content_length,avg_word_length,topics,...,year,month,day,hour,min,sec,weekends,img_number,link_number,title_length
0,0,0,-1,"<html><head><div class=""article-info""> <span c...",32,nasa grand challenge stop asteroid from destro...,there may killer asteroid head earth nasa deci...,583,5.214905,asteroid asteroids challenge earth space u s w...,...,2013,6,19,15,4,30,0,1,22,60
1,1,1,1,"<html><head><div class=""article-info""><span cl...",28,google new open sourc patent pledge We won t s...,googl took stand sort patent lawsuit theater t...,309,5.032787,app software google open source opn pledge pat...,...,2013,3,28,17,40,55,0,2,18,74
2,2,2,1,"<html><head><div class=""article-info""><span cl...",7,ballin nfl draft pick get choos their own walk...,you v spend countless hour train american foot...,1360,4.750225,entertainment nfl nfl draft sports televis,...,2014,5,7,19,15,20,0,2,11,68
3,3,3,-1,"<html><head><div class=""article-info""><span cl...",31,cameraperson fail deliv slapstick laugh,tire old sport fail news fails let cameraperso...,476,4.841727,sports video videos watercool,...,2013,10,11,2,26,50,0,1,13,43
4,4,4,-1,"<html><head><div class=""article-info""><span cl...",7,nfl star help young fan prove friendship with ...,At pounds all pro nfl star j j watt terror for...,1937,5.089650,entertainment instagram instagram video nfl sport,...,2014,4,17,3,31,43,0,52,16,61


'df_test (11847, 20)'

,index,Id,Page content,data-channel,title,raw_content,content_length,avg_word_length,topics,author,year,month,day,hour,min,sec,weekends,img_number,link_number,title_length
0,0,27643,"<html><head><div class=""article-info""><span cl...",5,soccer star get twitter death threat after tac...,note humanity one direct fandom ain t nothin m...,622,5.220114,entertainment music on direction soccer sport,243,2013,9,9,19,47,2,0,1,30,74
1,1,27644,"<html><head><div class=""article-info""><span cl...",25,googl glass get accessori store,shortli announc hardwar upgrad googl glass own...,149,4.739437,gadgets glass google googl glass googl glass tech,263,2013,10,31,9,25,2,0,3,13,36
2,2,27645,"<html><head><div class=""article-info""><span cl...",2,ouya game consol alreadi sold out amazon,well quick just hour go sale u s canada uk ouy...,168,5.018293,amazon amazon kindle business gam,278,2013,6,25,12,54,54,0,2,13,46
3,3,27646,"<html><head><div class=""article-info""><span cl...",6,two ferns mock oscar nomine,between two ferns oscar buzz edit part zach ga...,162,5.581699,two ferns movies th oscars oscar,211,2013,2,13,3,30,21,0,1,15,40
4,4,27647,"<html><head><div class=""article-info""><span cl...",5,sniper trailer look like eastwood may Be back ...,ever sinc the hurt locker seem like military t...,225,5.041096,american sniper awards bradley cooper clint ea...,142,2014,10,3,1,34,54,0,1,10,70


# String to Matrix & Dimension Reduction

In [8]:
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.manifold import TSNE, LocallyLinearEmbedding, SpectralEmbedding
from sklearn.decomposition import LatentDirichletAllocation

class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [9]:
from joblib import Parallel, delayed

def str2mat(df, txt2mat, dim_red):
    proc_pipe = Pipeline([txt2mat, ('dtrans', DenseTransformer()), dim_red])
    titles = proc_pipe.fit_transform(df['title'].values)
    titles = pd.DataFrame(titles)
    print(f'Titles Shape: {titles.shape}')

    raw_contents = proc_pipe.fit_transform(df['raw_content'].values)
    raw_contents = pd.DataFrame(raw_contents)
    print(f'Raw Contents Shape: {raw_contents.shape}')

    df_cat = pd.concat([df, titles, raw_contents], axis=1, ignore_index=True)
    print(f'X_train Shape: {df_cat.shape}')
    display(df_cat.head(5))

    return df_cat

def save_trans_task0(vals, tr_te_type, tm):
    return Parallel(n_jobs=len(dim_red))(delayed(save_trans_task1)(
        vals, tr_te_type, tm, dr) for dr in dim_red)

def save_trans_task1(vals, tr_te_type, tm, dr):
    # vals = [('title', title_vals), ('content', raw_content_vals)]

    return Parallel(n_jobs=len(vals))(delayed(save_trans_task)(
        val[1], val[0], tr_te_type, tm, dr) for val in vals)

def save_trans_task(vals, val_name, tr_te_type, tm ,dr):
    proc_pipe = Pipeline([tm, ('dtrans', DenseTransformer()), dr])
    ts = proc_pipe.fit_transform(vals)
    df = pd.DataFrame(ts, columns=[f'{val_name}_i' for i in range(ts.shape[1])])
    save_to(df, f'{tr_te_type}/{val_name}_{tm[0]}_{dr[0]}_{tr_te_type}.csv')

    print(f'{val_name}_{tm[0]}_{dr[0]}_{tr_te_type}: shape{df.shape}')
    display(df.head(5))

    return f'{val_name}_{tm[0]}_{dr[0]}_{tr_te_type}: shape{df.shape}', df.head(5)

def save_trans(df, tr_te_type, txt2mat, dim_red):
#     vals = [('title', df['title'].values), ('content', df['raw_content'].values)]
    vals = [('title', df['title'].values), ('content', df['raw_content'].values), ('topic', df['topics'].values)]

    return Parallel(n_jobs=len(txt2mat))(delayed(save_trans_task0)(
        vals, tr_te_type, tm) for tm in txt2mat)

def save_trans_custom(vals, tr_te_type, txt2mat, dim_red):
#     vals = [('title', df['title'].values), ('content', df['raw_content'].values), ('topic', df['topics'].values)]

    return Parallel(n_jobs=len(txt2mat))(delayed(save_trans_task0)(
        vals, tr_te_type, tm) for tm in txt2mat)

In [10]:
def check_null(df):
    # print(X_train.isna().sum())
    df = df.fillna(method='ffill')
    print(f'{df.isna().sum()}')
    print(f'Total Null: {df.isna().sum().sum()}')
    print(f'{df.shape}')
    display(df.head(5))
    
    return df

In [13]:
df_train = read_csv('train/new/preprocessed_train.csv')
df_train = check_null(df_train)

index              0
Id                 0
Popularity         0
Page content       0
data-channel       0
title              0
raw_content        0
content_length     0
avg_word_length    0
topics             0
author             0
year               0
month              0
day                0
hour               0
min                0
sec                0
weekends           0
img_number         0
link_number        0
title_length       0
dtype: int64
Total Null: 0
(27643, 21)


,index,Id,Popularity,Page content,data-channel,title,raw_content,content_length,avg_word_length,topics,...,year,month,day,hour,min,sec,weekends,img_number,link_number,title_length
0,0,0,-1,"<html><head><div class=""article-info""> <span c...",32,nasa grand challenge stop asteroid from destro...,there may killer asteroid head earth nasa deci...,583,5.214905,asteroid asteroids challenge earth space u s w...,...,2013,6,19,15,4,30,0,1,22,60
1,1,1,1,"<html><head><div class=""article-info""><span cl...",28,google new open sourc patent pledge We won t s...,googl took stand sort patent lawsuit theater t...,309,5.032787,app software google open source opn pledge pat...,...,2013,3,28,17,40,55,0,2,18,74
2,2,2,1,"<html><head><div class=""article-info""><span cl...",7,ballin nfl draft pick get choos their own walk...,you v spend countless hour train american foot...,1360,4.750225,entertainment nfl nfl draft sports televis,...,2014,5,7,19,15,20,0,2,11,68
3,3,3,-1,"<html><head><div class=""article-info""><span cl...",31,cameraperson fail deliv slapstick laugh,tire old sport fail news fails let cameraperso...,476,4.841727,sports video videos watercool,...,2013,10,11,2,26,50,0,1,13,43
4,4,4,-1,"<html><head><div class=""article-info""><span cl...",7,nfl star help young fan prove friendship with ...,At pounds all pro nfl star j j watt terror for...,1937,5.089650,entertainment instagram instagram video nfl sport,...,2014,4,17,3,31,43,0,52,16,61


# Train Data Dimension Reduction

In [92]:
# df_train = df_train[:1000]

In [18]:
txt2mat = [
           ('count', CountVectorizer(ngram_range=(1, 2), max_features=1000)),
           ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=1000))
           ]

dim_red = [
             ('lda', LatentDirichletAllocation(n_components=10, n_jobs = paral_procs)),
             ('lda100', LatentDirichletAllocation(n_components=100, n_jobs = paral_procs)),
             ('tsne', TSNE(n_components=3)),
#              ('lle', LocallyLinearEmbedding(n_components=8, n_neighbors=10, method='modified', n_jobs = paral_procs)), # modified LLE requires n_neighbors >= n_components
#              ('se', SpectralEmbedding(n_components=5, n_jobs=paral_procs)),
          ]

save_trans(df_train, 'train', txt2mat, dim_red)

ValueError: Error in determining null-space with ARPACK. Error message: 'Factor is exactly singular'. Note that method='arpack' can fail when the weight matrix is singular or otherwise ill-behaved.  method='dense' is recommended. See online documentation for more information.

# Test Data Dimension Reduction

In [17]:
df_test = read_csv('test/new/preprocessed_test.csv')
df_test = check_null(df_test)

index              0
Id                 0
Page content       0
data-channel       0
title              0
raw_content        0
content_length     0
avg_word_length    0
topics             0
author             0
year               0
month              0
day                0
hour               0
min                0
sec                0
weekends           0
img_number         0
link_number        0
title_length       0
dtype: int64
Total Null: 0
(11847, 20)


,index,Id,Page content,data-channel,title,raw_content,content_length,avg_word_length,topics,author,year,month,day,hour,min,sec,weekends,img_number,link_number,title_length
0,0,27643,"<html><head><div class=""article-info""><span cl...",5,soccer star get twitter death threat after tac...,note humanity one direct fandom ain t nothin m...,622,5.220114,entertainment music on direction soccer sport,243,2013,9,9,19,47,2,0,1,30,74
1,1,27644,"<html><head><div class=""article-info""><span cl...",25,googl glass get accessori store,shortli announc hardwar upgrad googl glass own...,149,4.739437,gadgets glass google googl glass googl glass tech,263,2013,10,31,9,25,2,0,3,13,36
2,2,27645,"<html><head><div class=""article-info""><span cl...",2,ouya game consol alreadi sold out amazon,well quick just hour go sale u s canada uk ouy...,168,5.018293,amazon amazon kindle business gam,278,2013,6,25,12,54,54,0,2,13,46
3,3,27646,"<html><head><div class=""article-info""><span cl...",6,two ferns mock oscar nomine,between two ferns oscar buzz edit part zach ga...,162,5.581699,two ferns movies th oscars oscar,211,2013,2,13,3,30,21,0,1,15,40
4,4,27647,"<html><head><div class=""article-info""><span cl...",5,sniper trailer look like eastwood may Be back ...,ever sinc the hurt locker seem like military t...,225,5.041096,american sniper awards bradley cooper clint ea...,142,2014,10,3,1,34,54,0,1,10,70


In [ ]:
txt2mat = [
           ('count', CountVectorizer(ngram_range=(1, 2), max_features=1000)),
           ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=1000))
           ]

dim_red = [
           ('lda', LatentDirichletAllocation(n_components=10, n_jobs = paral_procs)),
           ('lda100', LatentDirichletAllocation(n_components=100, n_jobs = paral_procs)),
#            ('lleH', LocallyLinearEmbedding(n_components=20, n_neighbors=250, method='hessian', n_jobs = paral_procs)), # for method='hessian', n_neighbors must be greater than [n_components * (n_components + 3) / 2]
#            ('lle', LocallyLinearEmbedding(n_components=8, n_neighbors=10, method='modified', n_jobs = paral_procs)), # modified LLE requires n_neighbors >= n_components
#            ('se', SpectralEmbedding(n_components=5, n_jobs=paral_procs)),
           ('tsne', TSNE(n_components=3))
          ]

save_trans(df_test, 'test', txt2mat, dim_red)

In [14]:
def save_tf(vals, tr_te_type):
#     vals = [('title', df['title'].values), ('content', df['raw_content'].values), ('topic', df['topics'].values)]
    tm = 'tfidf'
#     tr_te_type = 'train'

    for val in vals:
        tf = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)
        ts = tf.fit_transform(val[1]).toarray()
        save_to(pd.DataFrame(ts), f'{tr_te_type}/{val[0]}_{tm}_{tr_te_type}.csv')

        print(f'{val[0]}_{tm}_{tr_te_type}: shape{ts.shape}')
        display(pd.DataFrame(ts).head(5))

In [ ]:
tt_df = df_train.iloc[:100, :]
save_tf([('title', df_train['title'].values), 
         ('content', df_train['raw_content'].values), 
         ('topic', df_train['topics'].values)], 'train')

In [ ]:
tt_df = df_test.iloc[:100, :]
save_tf([('title', df_test['title'].values), 
         ('content', df_test['raw_content'].values), 
         ('topic', df_test['topics'].values)], 'test')